In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import shutil
import numpy as np
import thuner.data as data
import thuner.option as option
import thuner.analyze as analyze
import thuner.parallel as parallel
import thuner.visualize as visualize
import thuner.track as track
import thuner.default as default

notebook_name = "gridrad_demo.ipynb"


Welcome to the Thunderstorm Event Reconnaissance (THUNER) package 
v0.0.11! This is a placeholder version of the package and is not
yet functional. Please visit github.com/THUNER-project/THUNER for 
examples, and to report issues or contribute.

THUNER is a flexible toolkit for performing multi-feature detection, 
tracking, tagging and analysis of events within meteorological datasets. 
The intended application is to convective weather events. For examples 
and instructions, see github.com/THUNER-project/THUNER. If you use this 
package in your research, consider citing the following papers;

Short et al. (2023), doi: 10.1175/MWR-D-22-0146.1
Raut et al. (2021), doi: 10.1175/JAMC-D-20-0119.1
Fridlind et al. (2019), doi: 10.5194/amt-12-2979-2019
Whitehall et al. (2015), doi: 10.1007/s12145-014-0181-3
Dixon and Wiener (1993), doi: 10.1175/1520-0426(1993)010<0785:TTITAA>2.0.CO;2
Leese et al. (1971), doi: 10.1175/1520-0450(1971)010<0118:AATFOC>2.0.CO;2



In [2]:
# Parent directory for saving outputs
base_local = Path.home() / "THUNER_output"
year=2010
event_directories = data.gridrad.get_event_directories(year, base_local=base_local)
event_directory = event_directories[0]
start, end, event_start = data.gridrad.get_event_times(event_directory)
# start = "2010-01-20T23:30:00"
# end = "2010-01-21T00:30:00"

output_parent = base_local / f"runs/gridrad/gridrad_demo"

In [3]:
# if output_parent.exists():
# shutil.rmtree(output_parent)
options_directory = output_parent / "options"

# Create and save the dataset options
times_dict = {"start": start, "end": end}
gridrad_dict = {"event_start": event_start}
gridrad_options = data.gridrad.GridRadSevereOptions(**times_dict, **gridrad_dict)
era5_dict = {"latitude_range": [27, 39], "longitude_range": [-102, -89]}
era5_pl_options = data.era5.ERA5Options(**times_dict, **era5_dict)
era5_dict.update({"data_format": "single-levels"})
era5_sl_options = data.era5.ERA5Options(**times_dict, **era5_dict)
datasets = [gridrad_options, era5_pl_options, era5_sl_options]
data_options = option.data.DataOptions(datasets=datasets)
data_options.to_yaml(options_directory / "data.yml")

# Create and save the grid_options dictionary
kwargs = {"name": "geographic", "regrid": False, "altitude_spacing": None}
kwargs.update({"geographic_spacing": None})
grid_options = option.grid.GridOptions(**kwargs)
grid_options.to_yaml(options_directory / "grid.yml")

# Create the track_options dictionary
track_options = default.track(dataset="gridrad")
# Modify the default options for gridrad. Because grids so large we now use a distinct
# global flow box for each object.
track_options.levels[1].objects[0].tracking.global_flow_margin = 70
track_options.levels[1].objects[0].tracking.unique_global_flow = False
# If testing, remove the profile and tag attributes
# track_options.levels[1].objects[0].attributes["mcs"].pop("profile")
# track_options.levels[1].objects[0].attributes["mcs"].pop("tag")
track_options.to_yaml(options_directory / "track.yml")

visualize_options = None

2025-03-02 18:06:14,140 - thuner.data.gridrad - INFO - Generating era5 filepaths.
2025-03-02 18:06:14,144 - thuner.data.era5 - INFO - Generating era5 filepaths.
2025-03-02 18:06:14,146 - thuner.data.era5 - INFO - Generating era5 filepaths.
2025-03-02 18:06:14,172 - thuner.option.grid - WARNING - altitude_spacing not specified. Will attempt to infer from input.
2025-03-02 18:06:14,172 - thuner.option.grid - WARNING - shape not specified. Will attempt to infer from input.


In [4]:
times = data.utils.generate_times(data_options.dataset_by_name("gridrad"))
args = [times, data_options, grid_options, track_options, visualize_options]
parallel.track(*args, output_directory=output_parent)
# track.track(*args, output_directory=output_parent)

2025-03-02 18:06:14,303 - thuner.parallel - INFO - Beginning parallel tracking with 4 processes.


2025-03-02 18:06:22,168 - thuner.track.track - INFO - Beginning thuner tracking. Saving output to /home/ewan/THUNER_output/runs/gridrad/gridrad_demo/interval_0.
2025-03-02 18:06:22,372 - thuner.track.track - INFO - Beginning thuner tracking. Saving output to /home/ewan/THUNER_output/runs/gridrad/gridrad_demo/interval_1.
2025-03-02 18:06:22,925 - thuner.track.track - INFO - Beginning thuner tracking. Saving output to /home/ewan/THUNER_output/runs/gridrad/gridrad_demo/interval_2.
2025-03-02 18:06:23,016 - thuner.track.track - INFO - Beginning thuner tracking. Saving output to /home/ewan/THUNER_output/runs/gridrad/gridrad_demo/interval_3.
2025-03-02 18:06:23,230 - thuner.track.track - INFO - Processing 2010-01-20T20:20:00.
2025-03-02 18:06:23,233 - thuner.data.gridrad - INFO - Updating gridrad dataset for 2010-01-20T20:20:00.
2025-03-02 18:06:23,233 - thuner.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100120T202000Z.nc
2025-03-02 18:06:23,251 - thuner.track.track -

In [5]:
analysis_options = analyze.mcs.AnalysisOptions()
analyze.mcs.process_velocities(output_parent, profile_dataset="era5_pl")
analyze.mcs.quality_control(output_parent, analysis_options)
# analyze.mcs.classify_all(output_parent, analysis_options)

2025-03-02 18:10:35,751 - thuner.option.grid - WARNING - altitude_spacing not specified. Will attempt to infer from input.
2025-03-02 18:10:35,752 - thuner.option.grid - WARNING - shape not specified. Will attempt to infer from input.


2025-03-02 18:10:36,134 - thuner.option.grid - WARNING - altitude_spacing not specified. Will attempt to infer from input.
2025-03-02 18:10:36,135 - thuner.option.grid - WARNING - shape not specified. Will attempt to infer from input.


In [6]:
figure_name = f"mcs_gridrad_{event_start.replace('-', '')}"
kwargs = {"name": figure_name, "style": "presentation"}
kwargs.update({"attributes": ["velocity", "offset"]})
figure_options = option.visualize.HorizontalAttributeOptions(**kwargs)
start_time = np.datetime64(start)
end_time = np.datetime64(end)
args = [output_parent, start_time, end_time, figure_options]
args_dict = {"parallel_figure": True, "dt": 7200, "by_date": False, "num_processes": 4}
visualize.attribute.mcs_series(*args, **args_dict)

2025-03-02 18:10:36,773 - thuner.option.grid - WARNING - altitude_spacing not specified. Will attempt to infer from input.
2025-03-02 18:10:36,774 - thuner.option.grid - WARNING - shape not specified. Will attempt to infer from input.
2025-03-02 18:10:37,308 - thuner.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:00:00.000000000.
2025-03-02 18:10:39,137 - thuner.option.grid - WARNING - altitude_spacing not specified. Will attempt to infer from input.
2025-03-02 18:10:39,137 - thuner.option.grid - WARNING - shape not specified. Will attempt to infer from input.
2025-03-02 18:10:40,324 - thuner.visualize.attribute - INFO - Saving mcs_gridrad_20100120 figure for 2010-01-20T18:00:00.000000000.
2025-03-02 18:10:50,071 - thuner.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:10:00.000000000.
2025-03-02 18:10:50,071 - thuner.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:20:00.000000000.
2025-03-02 18:10:50,161 - thuner.visualize.attr